In [1]:
!pip install transformers

In [2]:
!pip install accelerate -U

In [3]:
!pip install torch transformers bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.6 MB/s eta 0:00:00:00:0100:01


In [4]:
!pip install --quiet bitsandbytes
!pip install --quiet --upgrade transformers
!pip install --quiet --upgrade accelerate
!pip install --quiet sentencepiece

In [5]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 9.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.7 MB/s eta 0:00:00


In [43]:
!pip install huggingface_hub

In [6]:
import pandas as pd
df = pd.read_csv('cricket_text_data.csv')
df.head()

,Filename,Context,Team_1,Team_2,Winner
0,1426261.json,Cricket match played on city Mohali on 2024-04...,Sunrisers Hyderabad (SH),Punjab Kings (PK),Sunrisers Hyderabad (SH)
1,1359507.json,Cricket match played on city Kolkata on 2023-0...,Chennai Super Kings (CSK),Kolkata Knight Riders (KKR),Chennai Super Kings (CSK)
2,392217.json,Cricket match played on city Kimberley on 2009...,Chennai Super Kings (CSK),Rajasthan Royals (RR),Chennai Super Kings (CSK)
3,1254112.json,Cricket match played on city Sharjah on 2021-1...,Mumbai Indians (MI),Delhi Capitals (DC),Delhi Capitals (DC)
4,829817.json,Cricket match played on city Mumbai on 2015-05...,Chennai Super Kings (CSK),Mumbai Indians (MI),Mumbai Indians (MI)


In [7]:
df['Winner'].value_counts()

Winner
Mumbai Indians (MI)                  142
Chennai Super Kings (CSK)            138
Kolkata Knight Riders (KKR)          129
Royal Challengers Bangalore (RCB)    114
Rajasthan Royals (RR)                110
Sunrisers Hyderabad (SH)              86
Kings XI Punjab (KXP)                 85
Delhi Daredevils (DD)                 67
Delhi Capitals (DC)                   45
Deccan Chargers (DC)                  29
Gujarat Titans (GT)                   28
Punjab Kings (PK)                     24
Lucknow Super Giants (LSG)            24
None (N)                              19
Gujarat Lions (GL)                    13
Pune Warriors (PW)                    12
Rising Pune Supergiant (RPS)          10
Royal Challengers Bengaluru (RCB)      7
Kochi Tuskers Kerala (KTK)             6
Rising Pune Supergiants (RPS)          5
Name: count, dtype: int64

In [8]:
def create_binary_label(row):
    if row['Winner'] == row['Team_1']:
        return 0
    elif row['Winner'] == row['Team_2']:
        return 1
    else:
        return -1

In [9]:
df['binary_label'] = df.apply(create_binary_label, axis=1)

In [10]:
df.head()

,Filename,Context,Team_1,Team_2,Winner,binary_label
0,1426261.json,Cricket match played on city Mohali on 2024-04...,Sunrisers Hyderabad (SH),Punjab Kings (PK),Sunrisers Hyderabad (SH),0
1,1359507.json,Cricket match played on city Kolkata on 2023-0...,Chennai Super Kings (CSK),Kolkata Knight Riders (KKR),Chennai Super Kings (CSK),0
2,392217.json,Cricket match played on city Kimberley on 2009...,Chennai Super Kings (CSK),Rajasthan Royals (RR),Chennai Super Kings (CSK),0
3,1254112.json,Cricket match played on city Sharjah on 2021-1...,Mumbai Indians (MI),Delhi Capitals (DC),Delhi Capitals (DC),1
4,829817.json,Cricket match played on city Mumbai on 2015-05...,Chennai Super Kings (CSK),Mumbai Indians (MI),Mumbai Indians (MI),1


In [11]:
df['binary_label'].value_counts()

binary_label
 0    547
 1    527
-1     19
Name: count, dtype: int64

In [12]:
df = df[df['binary_label'] != -1]

In [13]:
df['binary_label'].value_counts()

binary_label
0    547
1    527
Name: count, dtype: int64

In [14]:
data = df[['Context','binary_label']]
data = data[0:1093]
data.head()

,Context,binary_label
0,Cricket match played on city Mohali on 2024-04...,0
1,Cricket match played on city Kolkata on 2023-0...,0
2,Cricket match played on city Kimberley on 2009...,0
3,Cricket match played on city Sharjah on 2021-1...,1
4,Cricket match played on city Mumbai on 2015-05...,1


In [15]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [16]:
from transformers import BertTokenizer, BertForSequenceClassification
desired_vocab_size = 30522
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', vocab_size=desired_vocab_size)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2,)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/Users/pdp/Documents/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
model.resize_token_embeddings(len(tokenizer))

Embedding(30522, 768, padding_idx=0)

In [18]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [19]:
model = model.to(device)

In [20]:
X = list(data["Context"])
y = list(data["binary_label"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [21]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [22]:
# X_train_tokenized['attention_mask'][0]

In [24]:
len(X_train),len(X_val)

(859, 215)

In [25]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [26]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [27]:
# train_dataset[5]

In [28]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='binary')
    acc = accuracy_score(p.label_ids, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [32]:
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
new_model = "CreatedWinPredictor"
trainer.save_model(new_model)

In [ ]:
model_2 = BertForSequenceClassification.from_pretrained(new_model)
model_2.to(device)

In [ ]:
text = "who is going to win if match is played beteen RCB and SRH"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to(device)
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

### Hyper parameter tuning

In [33]:
import optuna
from transformers import TrainingArguments, Trainer

def model_init():
    return model

def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-4)
    per_device_train_batch_size = trial.suggest_categorical('per_device_train_batch_size', [8, 16, 32])
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
    warmup_steps = trial.suggest_int('warmup_steps', 0, 500)
    num_train_epochs = trial.suggest_int('num_train_epochs', 1, 5)
    
    training_args = TrainingArguments(
        output_dir="./results",
        learning_rate=learning_rate,
        per_device_train_batch_size=per_device_train_batch_size,
        weight_decay=weight_decay,
        warmup_steps=warmup_steps,
        num_train_epochs=num_train_epochs,
        evaluation_strategy="epoch",
    )

    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    eval_result = trainer.evaluate()
    return eval_result['eval_accuracy']

In [34]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=15)

[I 2024-05-27 17:06:42,938] A new study created in memory with name: no-name-7e50c471-3fa4-4193-9841-28dd1ba3d3b4
/tmp/ipykernel_10923/1257925144.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-4)
/tmp/ipykernel_10923/1257925144.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.642344,0.739535,0.724771,0.752381,0.738318
2,No log,0.544772,0.739535,0.724771,0.752381,0.738318
3,No log,0.423826,0.720930,1.000000,0.428571,0.600000
4,No log,0.416236,0.725581,0.794872,0.590476,0.677596


[I 2024-05-27 17:09:29,376] Trial 0 finished with value: 0.7255813953488373 and parameters: {'learning_rate': 1.374068100429785e-05, 'per_device_train_batch_size': 8, 'weight_decay': 0.0004377015845532701, 'warmup_steps': 257, 'num_train_epochs': 4}. Best is trial 0 with value: 0.7255813953488373.
/tmp/ipykernel_10923/1257925144.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-4)
/tmp/ipykernel_10923/1257925144.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.698590,0.488372,0.488372,1.000000,0.656250
2,No log,0.692888,0.511628,0.000000,0.000000,0.000000


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

[I 2024-05-27 17:33:57,905] Trial 12 finished with value: 0.7395348837209302 and parameters: {'learning_rate': 5.276101636897584e-05, 'per_device_train_batch_size': 8, 'weight_decay': 1.2476987404516102e-06, 'warmup_steps': 162, 'num_train_epochs': 5}. Best is trial 9 with value: 0.7395348837209302.
/tmp/ipykernel_10923/1257925144.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-4)
/tmp/ipykernel_10923/1257925144.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-package

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.574055,0.739535,0.724771,0.752381,0.738318
2,No log,0.579279,0.734884,0.714286,0.761905,0.737327
3,No log,0.560863,0.744186,0.695312,0.847619,0.763948
4,No log,0.537662,0.739535,0.656051,0.980952,0.786260
5,0.548700,0.605082,0.753488,0.691176,0.895238,0.780083


[I 2024-05-27 17:37:08,323] Trial 13 finished with value: 0.7534883720930232 and parameters: {'learning_rate': 3.5022538810558085e-05, 'per_device_train_batch_size': 8, 'weight_decay': 3.766198818443119e-05, 'warmup_steps': 466, 'num_train_epochs': 5}. Best is trial 13 with value: 0.7534883720930232.
/tmp/ipykernel_10923/1257925144.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-4)
/tmp/ipykernel_10923/1257925144.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packag

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.528939,0.734884,0.684615,0.847619,0.757447
2,No log,0.570087,0.734884,0.714286,0.761905,0.737327
3,No log,0.585420,0.748837,0.683453,0.904762,0.778689
4,No log,0.595721,0.730233,0.691057,0.809524,0.745614


[I 2024-05-27 17:39:33,535] Trial 14 finished with value: 0.7302325581395349 and parameters: {'learning_rate': 3.1217080951366144e-05, 'per_device_train_batch_size': 8, 'weight_decay': 9.249648961442178e-05, 'warmup_steps': 438, 'num_train_epochs': 4}. Best is trial 13 with value: 0.7534883720930232.


In [35]:
best_params = study.best_params
print(best_params)

{'learning_rate': 3.5022538810558085e-05, 'per_device_train_batch_size': 8, 'weight_decay': 3.766198818443119e-05, 'warmup_steps': 466, 'num_train_epochs': 5}


In [29]:
best_params = {'learning_rate': 3.5022538810558085e-05, 'per_device_train_batch_size': 8, 'weight_decay': 3.766198818443119e-05, 'warmup_steps': 466, 'num_train_epochs': 5}

In [30]:
training_args = TrainingArguments(
    output_dir="output",
    learning_rate=best_params['learning_rate'],
    per_device_train_batch_size=best_params['per_device_train_batch_size'],
    weight_decay=best_params['weight_decay'],
    warmup_steps=best_params['warmup_steps'],
    num_train_epochs=best_params['num_train_epochs'],
    evaluation_strategy="epoch", 
    save_strategy="epoch", 
    logging_dir="logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset, 
    compute_metrics=compute_metrics 
)

/Users/pdp/Documents/anaconda3/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [31]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.698400,0.653935,0.627907,0.662338,0.485714,0.560440
2,0.574000,0.564599,0.697674,0.685185,0.704762,0.694836
3,0.466300,0.421148,0.734884,1.000000,0.457143,0.627451
4,0.447100,0.412109,0.697674,0.617647,1.000000,0.763636
5,0.328500,0.400864,0.697674,0.704082,0.657143,0.679803


TrainOutput(global_step=540, training_loss=0.523768072658115, metrics={'train_runtime': 632.2722, 'train_samples_per_second': 6.793, 'train_steps_per_second': 0.854, 'total_flos': 498816422082600.0, 'train_loss': 0.523768072658115, 'epoch': 5.0})

In [32]:
eval_result = trainer.evaluate()
eval_result

{'eval_loss': 0.4008639454841614,
 'eval_accuracy': 0.6976744186046512,
 'eval_precision': 0.7040816326530612,
 'eval_recall': 0.6571428571428571,
 'eval_f1': 0.6798029556650246,
 'eval_runtime': 8.8143,
 'eval_samples_per_second': 24.392,
 'eval_steps_per_second': 3.063,
 'epoch': 5.0}

In [62]:
gen_model_name = "ipl_win_prediction_model"

In [63]:
trainer.save_model(gen_model_name)

In [64]:
tokenizer.save_pretrained(gen_model_name)

('ipl_win_prediction_model/tokenizer_config.json',
 'ipl_win_prediction_model/special_tokens_map.json',
 'ipl_win_prediction_model/vocab.txt',
 'ipl_win_prediction_model/added_tokens.json')

In [65]:
# model.save_pretrained(output)

In [33]:
trainer.save_model("final_win_prediction_model")

In [66]:
# trainer.save_model("final_win_prediction_model", tokenizer=tokenizer)

### Predicting win from generated model

In [79]:
hf_model_name = "pdp19/IPL_Win_Predictor"

In [80]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
tokenizer = BertForSequenceClassification.from_pretrained(hf_model_name)
personal_model = BertForSequenceClassification.from_pretrained(hf_model_name)

In [81]:
personal_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [87]:
input_text = "Who is going to win a match between SRH and KKR if it is played on Chennai?"
# inputs = tokenizer(input_text,).to(device)
# outputs = personal_model(**inputs)
# predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
# predictions = predictions.cpu().detach().numpy()
# predictions

## From Local

In [75]:
my_model = BertForSequenceClassification.from_pretrained(gen_model_name)
my_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [78]:
input_text = "Who is going to win a match between SRH and KKR if it is played on Chennai?"
inputs = tokenizer(input_text,padding = True, truncation = True, return_tensors='pt').to(device)
outputs = my_model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

array([[0.45634562, 0.5436544 ]], dtype=float32)

### Push it to Hugging face